<a href="https://colab.research.google.com/github/Yahred/evolutionary-computation/blob/main/CGXV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de rutas a través de algoritmos genéticos

Se cargan módulos necesarios

In [9]:
pip install deap gmaps==0.9.0 googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=8e83b801533ff0de177659a75cdfb16cc60d604bca20501a510edea412f3f5c6
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [12]:
import json
import math
import random
import folium
import gmaps
import gmaps.datasets
import googlemaps
import collections

import matplotlib.pyplot as plt
import numpy as np

from google.colab import output
from deap import base, creator, tools, algorithms
from urllib.request import urlopen
from IPython.display import display

Se cargan datos sobre las capitales

In [13]:
output.enable_custom_widget_manager()
collections.Iterable = collections.abc.Iterable

path = 'https://raw.githubusercontent.com/Yahred/evolutionary-computation/main/data/capitales.json'
archivo = urlopen(path).read()
data = json.loads(archivo)

GOOGLE_API_KEY="API_KEY"
CAPITALES = data['capitales']
COORDENADAS = data['coordenadas']
DISTANCIAS = data['distancias']
ESPACIOS = 8
CENTRO_EU = (54.6872, 25.2797)
ZOOM_MAPA = 4

gmaps.configure(api_key=GOOGLE_API_KEY)
maps_client = googlemaps.Client(key=GOOGLE_API_KEY)

Se calcula la longitud del cromosoma en base a las capitales y el número de espacios disponibles

In [5]:
LONGITUD_CROMOSOMA = 0
for i in range(ESPACIOS):
  LONGITUD_CROMOSOMA += math.ceil(math.log(len(CAPITALES) - i, 2))

LONGITUD_CROMOSOMA

28

In [6]:
CAPITALES

['Madrid',
 'París',
 'Berlín',
 'Roma',
 'Atenas',
 'Lisboa',
 'Varsovia',
 'Praga',
 'Ámsterdam',
 'Viena',
 'Bucarest',
 'Budapest']

In [7]:
COORDENADAS

{'Madrid': {'latitud': 40.416775, 'longitud': -3.70379},
 'París': {'latitud': 48.856613, 'longitud': 2.352222},
 'Berlín': {'latitud': 52.520008, 'longitud': 13.404954},
 'Roma': {'latitud': 41.902782, 'longitud': 12.496366},
 'Atenas': {'latitud': 37.98381, 'longitud': 23.727539},
 'Lisboa': {'latitud': 38.722252, 'longitud': -9.139337},
 'Varsovia': {'latitud': 52.229676, 'longitud': 21.012229},
 'Praga': {'latitud': 50.075538, 'longitud': 14.4378},
 'Ámsterdam': {'latitud': 52.366696, 'longitud': 4.89454},
 'Viena': {'latitud': 48.208174, 'longitud': 16.373819},
 'Bucarest': {'latitud': 44.426767, 'longitud': 26.102538},
 'Budapest': {'latitud': 47.497913, 'longitud': 19.040236}}

# Obtención de distancias de las capitales

Realizamos una consulta a la API de Google Maps para obtener la distancia en automovil de cada capital y llenar una matriz con ellas

In [40]:
DISTANCIAS = {}
for (origen, coord_origen) in COORDENADAS.items():
  for (destino, coord_destino) in COORDENADAS.items():
      result = maps_client.distance_matrix(origins=(coord_origen['latitud'], coord_origen['longitud']),
                                       destinations=(coord_destino['latitud'], coord_destino['longitud']))
      distancia = result['rows'][0]['elements'][0]['distance']['value'] / 1000
      if not origen in DISTANCIAS:
        DISTANCIAS[origen] = {}

      DISTANCIAS[origen][destino] = distancia

In [41]:
DISTANCIAS

{'Madrid': {'Madrid': 0.0,
  'París': 1275.418,
  'Berlín': 2320.479,
  'Roma': 1958.839,
  'Atenas': 3719.273,
  'Lisboa': 625.192,
  'Varsovia': 2856.34,
  'Praga': 2239.892,
  'Ámsterdam': 1777.502,
  'Viena': 2381.267,
  'Bucarest': 3385.092,
  'Budapest': 2563.052},
 'París': {'Madrid': 1274.873,
  'París': 0.0,
  'Berlín': 1054.993,
  'Roma': 1457.603,
  'Atenas': 2876.317,
  'Lisboa': 1737.323,
  'Varsovia': 1590.854,
  'Praga': 1030.926,
  'Ámsterdam': 512.015,
  'Viena': 1235.865,
  'Bucarest': 2310.176,
  'Budapest': 1484.472},
 'Berlín': {'Madrid': 2321.177,
  'París': 1055.781,
  'Berlín': 0.0,
  'Roma': 1503.376,
  'Atenas': 2336.706,
  'Lisboa': 2783.627,
  'Varsovia': 573.802,
  'Praga': 349.681,
  'Ámsterdam': 657.252,
  'Viena': 680.829,
  'Bucarest': 1698.23,
  'Budapest': 872.526},
 'Roma': {'Madrid': 1967.434,
  'París': 1458.37,
  'Berlín': 1502.955,
  'Roma': 0.0,
  'Atenas': 1272.138,
  'Lisboa': 2517.566,
  'Varsovia': 1783.175,
  'Praga': 1298.761,
  'Ámsterdam

Graficamos las capitales que tenemos de opción

In [42]:
def marcar_capitales():
  fig = gmaps.figure(center=CENTRO_EU, zoom_level=ZOOM_MAPA)

  marcadores = []
  for (capital, coord) in COORDENADAS.items():
    lat = coord['latitud']
    lng = coord['longitud']
    marcador = (lat, lng)
    marcadores.append(marcador)

  marcadores_layer = gmaps.marker_layer(marcadores)
  fig.add_layer(marcadores_layer)

  return fig

marcar_capitales()

Figure(layout=FigureLayout(height='420px'))

# Definición de la función de la evaluación

In [17]:
class Capital:
  def __init__(self, nombre: str, lat, lng) -> None:
    self.nombre = nombre
    self.lat = lat
    self.lng = lng

  def __str__(self):
    return f'{self.nombre} Latitud: {self.lat} Longitud: {self.lng}'

Definimos una función para gráficar el mapa de las capitales

In [18]:
def graficar_capitales(capitales: list[Capital]):
  fig = marcar_capitales()

  for i in range(1, len(capitales)):
    a = capitales[i - 1]
    b = capitales[i]

    coord_a = (a.lat, a.lng)
    coord_b = (b.lat, b.lng)

    ruta = gmaps.directions_layer(coord_a, coord_b)
    fig.add_layer(ruta)

  ultima_capital = capitales[-1]
  primera_capital = capitales[0]

  coord_ultima = (ultima_capital.lat, ultima_capital.lng)
  coord_primera = (primera_capital.lat, primera_capital.lng)

  ruta = gmaps.directions_layer(coord_ultima, coord_primera)
  fig.add_layer(ruta)

  return fig

Definimos una funcion para que nos devuelva la capital y sus coordenadas

In [19]:
def capital_factory(nombre: str):
  coordenadas = COORDENADAS[nombre]
  return Capital(nombre, coordenadas['latitud'], coordenadas['longitud'])

Definimos una funcion para calcular la distancia

In [20]:
def calcular_distancia(a: Capital, b: Capital):
  distancias = DISTANCIAS[a.nombre]
  return distancias[b.nombre]

Definimos una funcion que calcule el recorrido de las capitales

In [21]:
def calcular_recorrido(capitales: list[Capital]) -> float:
  distancia = 0
  ruta = capitales.copy()

  for i in range(1, len(ruta)):
    a = capitales[i - 1]
    b = capitales[i]
    distancia += calcular_distancia(a, b)

  distancia += calcular_distancia(ruta[-1], ruta[0])

  return distancia

In [22]:
def decode_ind(ind: str) -> list[Capital]:
  opciones = CAPITALES.copy()
  capitales = []
  bin_restante = ''.join([str(gen) for gen in ind])
  for i in range(ESPACIOS):
    n_bits = math.ceil(math.log(len(opciones), 2))
    segmento_bin = bin_restante[:n_bits]
    bin_restante = bin_restante[n_bits:]

    seleccion = int(segmento_bin, 2)

    if seleccion >= len(opciones):
      seleccion = len(opciones) - 1

    capital_seleccionada = capital_factory(opciones.pop(seleccion))
    capitales.append(capital_seleccionada)

  return capitales

Funcion fitness que nos devuelve la distancia

In [23]:
def fitness(ind: list[int]):
  capitales = decode_ind(ind)
  distancia = calcular_recorrido(capitales)
  return distancia,

# Definición de los individuos



In [24]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, LONGITUD_CROMOSOMA)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.4)
toolbox.register("select", tools.selTournament, tournsize=5)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)

In [25]:
ind = toolbox.individual()
ind

[1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1]

In [47]:
random.seed(64)
n_gen = 2000
initial_pop = 200

pop = toolbox.population(n=initial_pop)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.2, ngen=n_gen, halloffame=hof, verbose=True, stats=stats)

ganador = tools.selBest(pop, k=1)[0]

gen	nevals	avg    
0  	200   	12623.1
1  	128   	11103.5
2  	128   	10169  
3  	125   	9525.71
4  	140   	9506.64
5  	138   	8513.69
6  	124   	8415.22
7  	156   	7856.16
8  	140   	7608.44
9  	147   	7932.79
10 	145   	7318.5 
11 	137   	7599.07
12 	136   	8077   
13 	145   	7603.02
14 	144   	7632.3 
15 	131   	7484.45
16 	132   	7389.9 
17 	134   	7574.77
18 	143   	7354.51
19 	137   	7506.94
20 	138   	7175.42
21 	138   	7459.81
22 	133   	7522.71
23 	136   	7360.07
24 	133   	7929.12
25 	140   	7813.62
26 	118   	7313.26
27 	149   	7540.44
28 	127   	7118.76
29 	139   	7234.2 
30 	141   	7420.12
31 	145   	7610.55
32 	133   	7515.43
33 	136   	7347.72
34 	120   	7343.1 
35 	148   	7652.79
36 	142   	7906.31
37 	139   	7539.72
38 	143   	7557.12
39 	145   	7415.02
40 	138   	7354.24
41 	156   	7893.34
42 	125   	7706.21
43 	128   	7475   
44 	135   	7486.09
45 	138   	7463.78
46 	120   	7358.83
47 	120   	7427.07
48 	139   	7817.62
49 	142   	7439.01
50 	130   	7670.42
51 	138   	7

In [48]:
distancia, = fitness(ganador)
distancia

5866.23

In [49]:
capitales = decode_ind(ind)

In [50]:
graficar_capitales(capitales)

Figure(layout=FigureLayout(height='420px'))

# Algoritmo genético compacto

In [31]:
import random

class Individual:
    def __init__(self, chrom: list[int]) -> None:
        self.chrom = chrom
        self.fitness = None

    def __str__(self) -> str:
        return '{} Fitness: {}'.format(self.chrom, self.fitness)


def initialize_probs(num_genes: int) -> list[float]:
    return [0.5 for _ in range(num_genes)]


def create_individual(probs: list[float]):
    chrom = ['1' if random.uniform(0, 1) < prob else '0' for prob in probs]
    return Individual(''.join(chrom))


def compete(a: Individual, b: Individual, fitness: callable, fitness_min: bool):
    a.fitness = fitness(a.chrom)
    b.fitness = fitness(b.chrom)

    if a.fitness < b.fitness and fitness_min:
        return a, b

    if a.fitness > b.fitness and not fitness_min:
        return a, b

    return b, a


def adjust_probs(probs: list[float], winner: Individual, loser: Individual, step: int):
    new_probs = []

    for i in range(len(probs)):
        loser_gen = loser.chrom[i]
        winner_gen = winner.chrom[i]

        if winner_gen == loser_gen:
            new_probs.append(probs[i])
            continue

        if winner_gen == '0':
            new_probs.append(probs[i] - step)
            continue

        new_probs.append(probs[i] + step)
    return new_probs


def has_converged(probs: list[float], convergence_criteria: float):
    for prob in probs:
        diff = 1 - prob
        if diff > convergence_criteria:
            return False
    return True


def evolve(fitness: callable, num_genes: int, generations: int, step: int = 0.05, convergence_criteria=0.001, fitness_min=False):
    best = None
    probs = initialize_probs(num_genes)

    for _ in range(generations):
        a = create_individual(probs)
        b = create_individual(probs)

        winner, loser = compete(a, b, fitness, fitness_min)

        if not best:
            best = winner
        elif winner.fitness > best.fitness and not fitness_min:
            best = winner
        elif winner.fitness < best.fitness and fitness_min:
            best = winner

        probs = adjust_probs(probs, winner, loser, step)

        if has_converged(probs, convergence_criteria):
            break

    return best.chrom, best.fitness

Definimos la función de evaluación para el algoritmo genético compacto

In [33]:
def fitness_compact(ind: list[str]):
  capitales = decode_ind(ind)
  distancia = calcular_recorrido(capitales)
  return distancia

Ejecutamos la evolución

In [51]:
generations = 50000

best, distancia = evolve(fitness_compact, LONGITUD_CROMOSOMA, generations, step=0.0001, fitness_min=True)

In [52]:
best

'1010110001100010110001100110'

In [53]:
capitales_compacto = decode_ind(best)
calcular_recorrido(capitales_compacto)

5865.092000000001

In [54]:
graficar_capitales(capitales_compacto)

Figure(layout=FigureLayout(height='420px'))